<a href="https://colab.research.google.com/github/eTcilopp/geek_brains/blob/main/044_machine_learning/hw08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решающие деревья. Домашнее задание

## Что нужно сделать
Решите задачу классификации оттоковых клиентов телекома: для каждого клиента необходимо определить, перестанет он пользоваться услугами оператора в ближайшее время или нет.

In [2]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("https://gbcdn.mrgcdn.ru/uploads/asset/6057170/attachment/326b1a69e5645a4a2f66fdad2177bac7.csv")
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Сделайте первичную обработку данных:

1. Выбросите колонку State.

2. Колонки International plan и Voice mail plan закодируйте следующим образом: Yes замените на 1, а No на 0.

3. Переведите целевую переменную (Churn) в числа: True в 1, а False в 0.

In [9]:
df = df.drop(columns=['State'])
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [10]:
df['International plan'] = df['International plan'].map({'Yes': 1, 'No': 0})
df['Voice mail plan'] = df['Voice mail plan'].map({'Yes': 1, 'No': 0})
df['Churn'] = df['Churn'].map({True: 1, False: 0})

Выведите на экран первые строки изменённой таблицы, чтобы проверить, что все преобразования сработали.

In [11]:
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


Создайте матрицу объект-признак X и вектор с целевой переменной (Churn) y.

In [12]:
X = df.drop(columns=['Churn'])
y = df['Churn']

Проверьте баланс классов. Является ли выборка сбалансированной?

In [14]:
y.value_counts()

,count
Churn,
0,2850
1,483


Выборка не является сбалансированной

Разбейте данные на тренировочную и тестовую части, размер тестовой части — 30%.

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Для линейных моделей очень важно масштабирование признаков. Поэтому отмасштабируйте признаки при помощи [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), обучив метод на тренировочных данных и применив его и к трейну, и к тесту.

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

После масштабирования признаков X_train и X_test перестали быть объектами типа pd.DataFrame, это неудобно. Приведите их снова к pd.DataFrame.

In [18]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

Обучите на тренировочных данных с параметрами по умолчанию:

1) логистическую регрессию;

2) метод опорных векторов (с kernel='linear').

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

model_svc = SVC(kernel='linear')
model_svc.fit(X_train, y_train)

SVC(kernel='linear')

С помощью обученных моделей сделайте предсказание на тестовых данных.

In [20]:
from sklearn.metrics import roc_auc_score

y_pred_lr = model_lr.predict(X_test)
y_pred_svc = model_svc.predict(X_test)

roc_auc_score_lr = roc_auc_score(y_test, y_pred_lr)
roc_auc_score_svc = roc_auc_score(y_test, y_pred_svc)

print(f"ROC AUC для логистической регрессии: {roc_auc_score_lr}")
print(f"ROC AUC для метода опорных векторов: {roc_auc_score_svc}")

ROC AUC для логистической регрессии: 0.5903093406010559
ROC AUC для метода опорных векторов: 0.5


Теперь обучите решающее дерево с параметрами по умолчанию на тренировочных данных, сделайте предсказание на тесте. Используйте метрику для оценки.

In [21]:
from sklearn.tree import DecisionTreeClassifier

model_dtc = DecisionTreeClassifier()
model_dtc.fit(X_train, y_train)

y_pred_dtc = model_dtc.predict(X_test)

roc_auc_score_dtc = roc_auc_score(y_test, y_pred_dtc)

print(f"ROC AUC для решающего дерева: {roc_auc_score_dtc}")

ROC AUC для решающего дерева: 0.840295060831817


Нужно ли масшабировать признаки для обучения решающего дерева? Проверьте это, заново разбив данные на тренировочную и тестовую части (назовите новые полученные объекты новыми названиями), и проведите обучение, предсказание и оценку метрики до масштабирования и после.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model_dtc_no_scale = DecisionTreeClassifier()
model_dtc_no_scale.fit(X_train, y_train)

y_pred_dtc_no_scale = model_dtc_no_scale.predict(X_test)
roc_auc_score_dtc_no_scale = roc_auc_score(y_test, y_pred_dtc)

print(f"ROC AUC для решающего дерева БЕЗ масштабирования: {roc_auc_score_dtc_no_scale}")

ROC AUC для решающего дерева БЕЗ масштабирования: 0.840295060831817


In [23]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

model_dtc_scale = DecisionTreeClassifier()
model_dtc_scale.fit(X_train_s, y_train)

y_pred_dtc_scale = model_dtc_scale.predict(X_test_s)
roc_auc_score_dtc_scale = roc_auc_score(y_test, y_pred_dtc_scale)
print(f"ROC AUC для решающего дерева ПОСЛЕ масштабирования: {roc_auc_score_dtc_scale}")

ROC AUC для решающего дерева ПОСЛЕ масштабирования: 0.8333020538388101


Какая модель справилась с решением задачи?

Модель великолепно справилась с задачами. Масштабирование сделало результаты даже чуть хуже.